In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# https://huggingface.co/EleutherAI/polyglot-ko-5.8b
# https://colab.research.google.com/github/jagaldol/chat-foodie/blob/main/chatbot/fine-tuning/KoAlpaca_QLoRA_ChatFoodie_Model_FineTuning.ipynb#scrollTo=E0Nl5mWL0k2T

!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 23.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import Dataset, DatasetDict
import pandas as pd
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [4]:
# 저장한 모델 불러오고 적용해보기

In [5]:
model_id = "/content/drive/MyDrive/컨퍼런스/소제목/model/epoch300"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/36.8k [00:00<?, ?B/s]

model-00001-of-00013.safetensors:   0%|          | 0.00/926M [00:00<?, ?B/s]

model-00002-of-00013.safetensors:   0%|          | 0.00/952M [00:00<?, ?B/s]

...

model-00011-of-00013.safetensors:   0%|          | 0.00/952M [00:00<?, ?B/s]

model-00012-of-00013.safetensors:   0%|          | 0.00/948M [00:00<?, ?B/s]

model-00013-of-00013.safetensors:   0%|          | 0.00/515M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
p1text1 = '''
대학생 시절, 저는 AI 개발을 공부해보자는 다짐을 가지고 수상을 목표로 대회에 출전하였습니다.
AI 알고리즘 코드를 짜본 경험이 없었기 때문에 대회 진행에 어려움을 겪었습니다.
그리하여 AI 개발에 능숙하신 교수님과 선배님들에게 이메일을 보내 적극적으로 도움을 요청하였습니다.
또한, 대회와 관련된 교내의 모든 AI 서적과 깃허브의 코드를 공부하면서 코드를 완성하였습니다.
하지만 수상을 목표로 출전한 대회에서는 입상도 하지 못하였습니다.
하지만 실패에 낙담하지 않고 뭐든지 할 수 있다는 믿음으로 대회에 다시 출전하였습니다.
팀원을 모아 다양한 시각을 통해 문제를 극복하고, 정기적으로 교수님에게 첨삭을 받으면서 프로젝트의 방향성이 맞는 지를 확인했습니다.
1년 동안 같은 데이터로 꾸준히 연구한 결과를 논문으로 구체화하였고, 그 결과 대회에서 최우수상을 받을 수 있었습니다.
1년의 프로젝트에서 지치지 않고 좋은 성과를 만들어낸 경험을 토대로 SK텔레콤에서 추진력을 발휘하여 최고의 결과를 낼 수 있도록 기여하고 싶습니다.
'''

In [49]:
import re

def genTitle(text):
  system_prompt = "### System:\n입력받은 자기소개서에 대해서 강렬하고 핵심적인 소제목을 생성해줘.\n\n"
  prompt = f"{system_prompt}### 자기소개서: {text}\n\n### 소제목:\n"
  inputs = tokenizer(prompt, return_tensors="pt",return_token_type_ids=False).to("cuda")
  output = model.generate(**inputs, do_sample=True, temperature=0.9, top_p=0.75, max_new_tokens=30)
  opt = tokenizer.decode(output[0], skip_special_tokens=True)
  final = re.search(r"### 소제목:\s*(.*?)\s*\['EOS'\]", opt)

  if final:
    # 매치된 그룹에서 첫 번째 캡처된 결과 반환
    return final.group(1)
  else:
    # 매치가 없으면 다시 함수 부르기
    return genTitle(text)

In [38]:
p1text1 = '''
대학생 시절, 저는 AI 개발을 공부해보자는 다짐을 가지고 수상을 목표로 대회에 출전하였습니다.
AI 알고리즘 코드를 짜본 경험이 없었기 때문에 대회 진행에 어려움을 겪었습니다.
그리하여 AI 개발에 능숙하신 교수님과 선배님들에게 이메일을 보내 적극적으로 도움을 요청하였습니다.
또한, 대회와 관련된 교내의 모든 AI 서적과 깃허브의 코드를 공부하면서 코드를 완성하였습니다.
하지만 수상을 목표로 출전한 대회에서는 입상도 하지 못하였습니다.
하지만 실패에 낙담하지 않고 뭐든지 할 수 있다는 믿음으로 대회에 다시 출전하였습니다.
팀원을 모아 다양한 시각을 통해 문제를 극복하고, 정기적으로 교수님에게 첨삭을 받으면서 프로젝트의 방향성이 맞는 지를 확인했습니다.
1년 동안 같은 데이터로 꾸준히 연구한 결과를 논문으로 구체화하였고, 그 결과 대회에서 최우수상을 받을 수 있었습니다.
1년의 프로젝트에서 지치지 않고 좋은 성과를 만들어낸 경험을 토대로 SK텔레콤에서 추진력을 발휘하여 최고의 결과를 낼 수 있도록 기여하고 싶습니다.
'''

genTitle(p1text1)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'대학생 시절, 최고의 결과를 낸 성공적인 경험'

In [50]:
p1text2 = '''
전선 탐지 알고리즘 제작 프로젝트를 진행할 당시, AI 알고리즘이 촬영한 사진의 빛 번짐으로 인해 전선을 제대로 탐지하지 못하는 문제가 발생했습니다.
이를 해결하기 위한 회의 과정에서 빛 번짐을 제거할 수 있는 코드를 적용하여 데이터를 전처리 하자는 의견이 나왔습니다.
하지만 촬영한 데이터를 후처리하는 방식보다는 촬영 시에 빛 번짐이 덜한 종이를 뒤에 받쳐 찍으면 더 단순한 방식으로 문제를 해결할 수 있다고 생각했습니다.
그리하여 근처 문구점에서 하드보드지를 사 와서 빛 번짐이 발생하는지 테스트 해본 다음 해당 사진을 근거로 재촬영을 제안했습니다.
결과적으로 물체를 탐지하는 데 걸리는 시간을 최소화할 수 있었고 정확도가 20% 이상 향상 하는 성과를 달성할 수 있었습니다.
또한, 외주사에서 제시한 정확도를 달성하여 프로젝트를 진행할 수 있었습니다.
근본적으로 문제를 해결하려는 마음가짐으로 문제 해결을 위한 가장 효과적이고 단순한 방법을 고안하여 좋은 성과를 낼 수 있었습니다.
'''

genTitle(p1text2)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'빛 번짐 문제를 해결한 새로운 엔지니어'

In [51]:
p1text3 = '''
석사 과정 중 연구실 동기들과 논문들을 구현하며 공부하는 스터디를 진행하였습니다.
AI 분야는 오늘의 신기술이 언제 예전 기술이 될지 모를 만큼 발전과 연구가 빠르게 진행되는 분야이기에 새로운 논문이 나오면 그것을 빨리 구현해보고 특징과 장단점을 파악하는 것이 중요하다고 생각했습니다.
당장 최신 논문을 읽고 바로 구현하기에는 어려운 점이 많다고 판단해 먼저 인용 수가 많고 공식 구현 코드가 있는 논문들부터 구현하기로 결정하고 공부를 시작하였습니다.
하지만 공식 구현 코드를 참고하여 구현하는 것도 쉽지 않았습니다.
학부 시절 주로 java와 C, C++을 사용하였기에 Python의 여러 라이브러리와 프레임워크로 구현된 공식 구현 코드를 정확히 파악하기 어려웠습니다.
논문을 구현해보는 주목적은 논문을 정확히 이해하고 세부적인 동작을 파악하기 위함이었지만 코드를 따라잡고 이해하기 바빠 어느새 구현을 해보는 것에 급급하였습니다.
어려움에 봉착하였을 때 '기본적인 레퍼런스에 모든 해답이 있다'는 학부 코딩동아리 시절 선배들이 항상 했던 말이 떠올랐습니다.
그래서 먼저 Python의 기본 라이브러리와 PyTorch 와 같은 프레임워크를 세세히 파악하기로 결정하였습니다.
지금 당장 구현된 코드만을 이해하는 것보다 기본부터 다져 각각의 함수들이 어떤 동작을 하는지 정확하게 이해해야 다른 코드들도 수월히 구현할 수 있다고 판단했기 때문이었습니다.
기본적인 라이브러리와 프레임워크를 공부하고 나니 구현된 코드를 이해하는 것도 수월해졌고 다른 논문들 역시 큰 어려움 없이 구현할 수 있게 되었습니다.
또 논문을 구현하기 위해서는 논문 내용에 대한 정확한 이해가 바탕이 되어야 하지만 이해만 한다면 구현은 할 수 있겠다 하는 자신감도 생겼습니다.
이러한 경험을 토대로 항상 최신 기술을 공부하고 기본 레퍼런스에 충실한 AI 전문가로 거듭나고 싶습니다.
'''

genTitle(p1text3)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'라이브러리와 프레임워크를 활용해 목표를 달성하다'

In [52]:
p2text1 = '''
저의 가치관은 ‘작은 것이 모여 큰 것이 된다’입니다. 학부 시절, 작은 목표의 중요성을 깨닫고 문제 해결에 대해 한 걸음 쉽게 다가가게 된 경험이 있습니다.
서빙 아르바이트를 하던 중 메뉴판의 복잡한 레이아웃 때문에 손님들이 주문을 혼동하는 것을 확인 한 이후로, 그런 손님들이 오실 때마다 혼동한 메뉴들을 체크했습니다.
근무하는 4시간 동안 평균 6팀의 손님들이 세트메뉴 주문을 혼동했고 이것을 사장님께 전달해 직접 메뉴판의 정보 구조를 리디자인 했습니다.
메뉴의 그룹을 다시 구성했고 가장 헷갈려 하는 정보는 따로 분류해 직접 제작한 일러스트와 함께 재배치하여 강조했더니 메뉴 구성을 혼동하는 손님들이 평균 1팀 이하로 낮아졌습니다.
작은 시도라고 할 수 있지만 적극적인 자세로 문제점을 캐치하고 노력한 결과 수치로 확인할 수 있는 성과를 얻어 의미 있었던 경험이었고 큰 목표 뿐 아니라 주변의 작은 불편부터 바라보고 반응할 수 있는 계기가 되었습니다.
'''

genTitle(p2text1)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'사소한 것에서 시작한 변화의 성공'

In [53]:
p2text2 = '''
뜨거운 마음과 이성적인 사고를 갖춘 커뮤니케이터가 되겠습니다.
디자이너는 클라이언트는 물론 동료 디자이너, 개발자분들과 회의를 통해 디자인을 제작하기 때문에 상대방의 의견을 존중하고 수렴하는 자세가 필요하다고 생각합니다.
저는 적극적인 성격과 탁월한 커뮤니케이션 능력을 활용하여 팀 업무를 진행할 때 좋은 아이디어를 제시하고 효과적인 소통을 통해 디자인 방향성을 잡아가는 방법을 알고 있습니다.
또한 여러 개발언어에 대한 전반적인 지식을 가지고 있고 코딩 경험을 기반으로 하여 개발자분들과도 원활한 커뮤니케이션이 가능합니다.
이처럼 활발한 의사소통과 개발언어에 대한 이해로 사용자의 관점에서 어떤 구동과 디자인이 편리하고 직관적으로 보여지는 지를 고려해 웹과 모바일 디자인을 제작할 수 있습니다.
또한 여러 상품 및 서비스 기획 경험을 바탕으로 프로젝트를 진행할 때 디자인할 대상의 아이덴티티가 잘 반영되며 컨셉, 메세지를 확실히 전달하는 디자인을 제작할 수 있습니다.
제가 에이치엔그룹의 디자이너로서 일원이 된다면 팀에 조화를 이루며 협력하고 공동의 목표를 위해 완성된 디자인을 제작하는 디자이너가 되도록 하겠습니다.
트렌드를 놓치지 않는 세련된 디자인을 통해 디자인 경쟁력을 만들고 기획 단계부터 시장의 변화와 니즈를 철저히 수집하고 분석해 고객을 위한 가치창출에 매진하겠습니다.
'''

genTitle(p2text2)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'뜨거운 마음과 이성적인 사고를 갖춘 커뮤니케이터'

In [54]:
p2text3 = '''
저의 전공은 공간의 컨셉을 도출하여 연출, 설계하는 인테리어의 커리큘럼으로 이루어져 조형과 공간을 구성하는데 특화된 학과입니다.
가구 디자인, 인테리어 디자인, 개인 작업을 통해 일러스트, 포토샵 실력 또한 갖추어 왔습니다.
대학 재학 중, 직접 제작한 작품을 전시하는 과제·졸업 전시에서 졸업 전시의 디스플레이 팀장을 맡은 경험이 있습니다.
전시장 내부의 디스플레이를 전담하여 전시의 컨셉에 맞는 공간의 방향성을 제시하고 연출하였습니다.
좁은 전시장의 공간을 효율적으로 이용하기 위해 천장에 패브릭을 설치하였고 그 위에 프로젝터를 통해 영상을 재생하여 전시장의 분위기를 연출하였습니다.
또한, 각 작품이 가진 컨셉을 한눈에 담은 사진을 전시장 한편에 붙여 작품의 설명을 비주얼 적으로 나타냈습니다.
전시장 외부에는 전시의 방향성이 돋보이게 하는 조형물을 설치하여 전시를 홍보했습니다.
결과적으로 전년도보다 훨씬 전시 공간의 동선을 효과적으로 구성하였고 작품의 개성을 드러낼 수 있는 전시 공간을 연출할 수 있었습니다.
또한, 전년도보다 큰 홍보 효과로 많은 관람객을 유치하여 호평을 듣는 전시가 되었습니다.
전시를 연출하는 과정을 통해 공간을 효과적인 구성과 기획하는 법을 배웠고, 매력적인 공간을 연출하는 것에 관심을 두게 되었습니다.
이러한 경험이 이어져 VMD라는 직무에 매력을 느끼게 되었습니다.
영풍문고의 VMD로서 브랜드의 첫 인상이 될 수 있는 매장의 연출을 통해 고객의 오감을 만족하게 하는 공간을 기획하여 차별화된 브랜드 가치를 보여주겠습니다.
'''

genTitle(p2text3)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'공간의 컨셉을 도출하여 연출, 설계하는 인플루언서'